# 🐂 Sistema de Recomendación Multi-Objetivo 🐂
vamos a tratar de crear un sistema de recomendación de items usando el modelo"Collaborative-Filtering"

## Primero importamos las librerias

In [14]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
import time
from IPython.display import clear_output
from math import sqrt

## Cargamos los datos

In [3]:
df=pd.DataFrame()
train1=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_10000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train1.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train1)
print('train1')
train2=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_12000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train2.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train2)
print('train2')
train3=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_2000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train3.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train3)
print('train3')
train4=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_4000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train4.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train4)
print('train4')
train5=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_6000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train5.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train5)
print('train5')
train6=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/train_8000000.csv',
                   usecols=['aid','session','type'],dtype={"type": "category"})
train6.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(train6)
print('train6')
test1=pd.read_csv('/kaggle/input/otto-recommendation-input-files-in-csv-format/OTTO  Multi-Objective Recommender System/test.csv',
                  usecols=['aid','session','type'],dtype={"type": "category"})
test1.drop_duplicates(['aid','session','type'],inplace=True)
df=df.append(test1)
print('test1')

train1
train2
train3
train4
train5
train6
test1


In [ ]:
print('hello')

# Cambiamos los valores categóricos a 1,2,3
Para el df general y para el individual:

In [5]:
fdf=df
fdf['type']=fdf['type'].replace(['clicks','carts','orders'],[1,2,3])
fdf.rename(columns={'type':'rating'},inplace=True)
fdf.head()

KeyError: 'type'

In [8]:
test1.drop_duplicates(['aid','session','type'],inplace=True)
test1['type']=test1['type'].replace(['clicks','carts','orders'],[1,2,3])
test1.rename(columns={'type':'rating'},inplace=True)
test1

,session,aid,rating
0,12899779,59625,1
1,12899780,1142000,1
2,12899780,582732,1
3,12899780,973453,1
4,12899780,736515,1
...,...,...,...
6928118,14571577,1141710,1
6928119,14571578,519105,1
6928120,14571579,739876,1
6928121,14571580,202353,1


creamos la lista de sesiones para el test

In [9]:
#Lista de sessiones únicas
listsessions=np.unique(test1['session']).tolist()
print(len(listsessions)*3==5015409)

True


In [10]:
#filtrar los usuarios que no tienen relación
#Powerful Eloy Bull
peb=fdf[fdf['session'].isin(listsessions)]
peb=np.unique(peb['aid']).tolist()
peb=fdf[fdf['aid'].isin(peb)]
peb=fdf[fdf['session'].isin(peb["session"].values.tolist())]

In [15]:
bullbull=pd.DataFrame(columns=['session_type','labels'])
for o in listsessions[0:5]:
    print('N°:',listsessions.index(o),'de',len(listsessions))
    print(listsessions.index(o)/len(listsessions)*100,'%')
    #df del user que quiero averiguar
    user0=test1.loc[test1['session']==o]
    #df de los usuarios con compras en común
    snts=peb[peb['aid'].isin(user0["aid"].values.tolist())]
    #borro mi propio user
    snts=snts.loc[snts['session']!=o]
    if len(snts)==0:
        snts=user0
    user0.reset_index(drop=True, inplace=True)
    sntsGroup = snts.groupby(['session'])
    sntsGroup = sorted(sntsGroup,  key=lambda x: len(x[1]), reverse=True)
    sntsGroup = sntsGroup[0:20]
    pearsonCorrelationDict = {}
    #va a iterar por cada sntsgroup en el subset
    for name, group in sntsGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='aid')
        user0df = user0.sort_values(by='aid')
        #Get the N for the formula
        nRatings = len(group)
        #Get the review scores for the movies that they both have in common
        temp_df = user0[user0['aid'].isin(group['aid'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['rating'].tolist()
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0
    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['session'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    newtopuserating=snts[snts['session'].isin(topUsers['session'].head(10).tolist())].sort_values(by=['rating'],ascending=False)
    labelsvar=newtopuserating['aid'][:20].tolist()
    tdf=pd.DataFrame({'session_type':user0['session'][0],'labels':[' '.join(map(str, labelsvar))]})
    bullbull=bullbull.append(tdf)
    clear_output()
bullbull.to_csv('preproto.csv',index=False)
bullbull

N°: 3 de 1671803
0.0001794469802961234 %


ValueError: math domain error

In [ ]:
bullbull=pd.DataFrame(columns=['session_type','labels'])
for o in listsessions:
    print('N°:',listsessions.index(o),'de',len(listsessions))
    print(listsessions.index(o)/len(listsessions)*100,'%')
    #df del user que quiero averiguar
    user0=test1.loc[test1['session']==o]
    #df de los usuarios con compras en común
    snts=peb[peb['aid'].isin(user0["aid"].values.tolist())]
    #borro mi propio user
    snts=snts.loc[snts['session']!=o]
    if len(snts)==0:
        snts=user0
    user0.reset_index(drop=True, inplace=True)
    sntsGroup = snts.groupby(['session'])
    sntsGroup = sorted(sntsGroup,  key=lambda x: len(x[1]), reverse=True)
    sntsGroup = sntsGroup[0:20]
    pearsonCorrelationDict = {}
    #va a iterar por cada sntsgroup en el subset
    for name, group in sntsGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='aid')
        user0df = user0.sort_values(by='aid')
        #Get the N for the formula
        nRatings = len(group)
        #Get the review scores for the movies that they both have in common
        temp_df = user0[user0['aid'].isin(group['aid'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['rating'].tolist()
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0
    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['session'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    newtopuserating=snts[snts['session'].isin(topUsers['session'].head(10).tolist())].sort_values(by=['rating'],ascending=False)
    labelsvar=newtopuserating['aid'][:20].tolist()
    tdf=pd.DataFrame({'session_type':user0['session'][0],'labels':[' '.join(map(str, labelsvar))]})
    bullbull=bullbull.append(tdf)
    clear_output()
bullbull.to_csv('dia2.csv',index=False)
bullbull

In [ ]:
bullbull=pd.DataFrame(columns=['session_type','labels'])
for o in listsessions:
    print('N°:',listsessions.index(o),'de',len(listsessions))
    print(listsessions.index(o)/len(listsessions)*100,'%')
    #df del user que quiero averiguar
    user0=test1.loc[test1['session']==o]
    #df de los usuarios con compras en común
    snts=peb[peb['aid'].isin(user0["aid"].values.tolist())]
    #borro mi propio user
    snts=snts.loc[snts['session']!=o]
    if len(snts)==0:
        snts=user0
    user0.reset_index(drop=True, inplace=True)
    sntsGroup = snts.groupby(['session'])
    sntsGroup = sorted(sntsGroup,  key=lambda x: len(x[1]), reverse=True)
    sntsGroup = sntsGroup[0:20]
    pearsonCorrelationDict = {}
    #va a iterar por cada sntsgroup en el subset
    for name, group in sntsGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='aid')
        user0df = user0.sort_values(by='aid')
        #Get the N for the formula
        nRatings = len(group)
        #Get the review scores for the movies that they both have in common
        temp_df = user0[user0['aid'].isin(group['aid'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['rating'].tolist()
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0
    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['session'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    newtopuserating=snts[snts['session'].isin(topUsers['session'].head(10).tolist())].sort_values(by=['rating'],ascending=False)
    labelsvar=newtopuserating['aid'][:20].tolist()
    tdf=pd.DataFrame({'session_type':user0['session'][0],'labels':[' '.join(map(str, labelsvar))]})
    bullbull=bullbull.append(tdf)
    clear_output()
bullbull.to_csv('dia3.csv',index=False)
bullbull